#### MITA : HATE CRIME CLASSIFICATION

In [1]:
%matplotlib inline
from sklearn.preprocessing import StandardScaler
from sklearn import datasets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

DATA INGESTION

In [2]:
data = pd.read_csv('hate_crime.csv', encoding='Latin1')

/Users/melikaogarro/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


DATA PREPROCESSING / CLEANING

In [3]:
data.head(5)

,INCIDENT_ID,DATA_YEAR,ORI,PUB_AGENCY_NAME,PUB_AGENCY_UNIT,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,...,OFFENDER_RACE,OFFENDER_ETHNICITY,VICTIM_COUNT,OFFENSE_NAME,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,MULTIPLE_OFFENSE,MULTIPLE_BIAS
0,3015,1991,AR0040200,Rogers,NaN,City,AR,Arkansas,West South Central,South,...,White,NaN,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
1,3016,1991,AR0290100,Hope,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Simple Assault,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S
2,43,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
3,44,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
4,3017,1991,AR0350100,Pine Bluff,NaN,City,AR,Arkansas,West South Central,South,...,Black or African American,NaN,1,Aggravated Assault,1.0,Service/Gas Station,Anti-White,Individual,S,S


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219577 entries, 0 to 219576
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   INCIDENT_ID               219577 non-null  int64  
 1   DATA_YEAR                 219577 non-null  int64  
 2   ORI                       219577 non-null  object 
 3   PUB_AGENCY_NAME           219577 non-null  object 
 4   PUB_AGENCY_UNIT           6431 non-null    object 
 5   AGENCY_TYPE_NAME          219577 non-null  object 
 6   STATE_ABBR                219577 non-null  object 
 7   STATE_NAME                219577 non-null  object 
 8   DIVISION_NAME             219577 non-null  object 
 9   REGION_NAME               219577 non-null  object 
 10  POPULATION_GROUP_CODE     219577 non-null  object 
 11  POPULATION_GROUP_DESC     219577 non-null  object 
 12  INCIDENT_DATE             219577 non-null  object 
 13  ADULT_VICTIM_COUNT        51411 non-null   f

##### Due to the number of null entries or the little relevance to the project objectives, we decide to drop the following columns:
        INCIDENT ID
        ORI
        PUB_AGENCY_UNIT
        JUVENILE_VICTIM_COUNT
        ADULT_VICTIM_COUNT
        JUVENILE_OFFENDER_COUNT
        ADULT_OFFENDER COUNT
        OFFENDER_ETHNICITY

In [5]:
data.drop(columns=['INCIDENT_ID', 'ORI', 'PUB_AGENCY_UNIT', 'JUVENILE_VICTIM_COUNT', 'ADULT_VICTIM_COUNT', 'JUVENILE_OFFENDER_COUNT','ADULT_OFFENDER_COUNT','OFFENDER_ETHNICITY'], inplace=True)

Now, the Population Group Code and Population Group Description Relay the same data, Hence we only keep the code in order to differentiate between the groups.

In [6]:
data['POPULATION_GROUP_CODE'].unique()

array(['5', '6', '3', '2', '8D', '1B', '9B', '4', '9A', '7', '1C', '8C',
       '1A', '9D', '9C', '8B', '8A', '8E', '9E', '0'], dtype=object)

In [7]:
data['POPULATION_GROUP_DESC'].unique()

array(['Cities from 10,000 thru 24,999', 'Cities from 2,500 thru 9,999',
       'Cities from 50,000 thru 99,999',
       'Cities from 100,000 thru 249,999',
       'Non-MSA counties under 10,000',
       'Cities from 500,000 thru 999,999',
       'MSA counties from 25,000 thru 99,999',
       'Cities from 25,000 thru 49,999', 'MSA counties 100,000 or over',
       'Cities under 2,500', 'Cities from 250,000 thru 499,999',
       'Non-MSA counties from 10,000 thru 24,999',
       'Cities 1,000,000 or over', 'MSA counties under 10,000',
       'MSA counties from 10,000 thru 24,999',
       'Non-MSA counties from 25,000 thru 99,999',
       'Non-MSA counties 100,000 or over', 'Non-MSA State Police',
       'MSA State Police',
       'Possessions (Puerto Rico, Guam, Canal Zone, Virgin Islands, and American Samoa)'],
      dtype=object)

##### Reclassifying the Population Groups:
    1.1:Cities under 2,500
    1.2:Cities from 2,500 thru 9,999
    1.3:Cities from 10,000 thru 24,999
    1.4:Cities from 25,000 thru 49,999
    1.5:Cities from 50,000 thru 99,999
    1.6:Cities from 100,000 thru 249,999
    1.7:Cities from 250,000 thru 499,999
    1.8:Cities from 500,000 thru 999,999
    1.9:Cities 1,000,000 or over
    2.1:MSA counties under 10,000
    2.2:MSA counties from 10,000 thru 24,999
    2.3:MSA counties from 25,000 thru 99,999
    2.4:MSA counties 100,000 or over
    2.5:MSA State Police
    3.1:Non-MSA counties under 10,000
    3.2:Non-MSA counties from 10,000 thru 24,999
    3.3:Non-MSA counties from 25,000 thru 99,999
    3.4:Non-MSA counties 100,000 or over
    3.5:Non-MSA State Police
    4.0:Possessions (Puerto Rico, Guam, Canal Zone, Virgin Islands, and American Samoa)


In [8]:
data['POPULATION_GROUP_CODE']= data['POPULATION_GROUP_CODE'].map({'5':1.3 ,'6':1.2,'3':1.5,'2':1.6,'8D':3.1, '1B':1.8,'9B':2.3,'4':1.4,'9A':2.4, '7':1.1, '1C':1.7,'8C':3.2, '1A':1.9,'9D':2.1,'9C':2.2,'8B':3.3,'8A':3.4,'8E':3.5,'9E':2.5,'0':4.0})

In [9]:
data['POPULATION_GROUP_CODE']=data['POPULATION_GROUP_CODE'].astype(np.float64)

In [10]:
data.head(10)

,DATA_YEAR,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,POPULATION_GROUP_CODE,POPULATION_GROUP_DESC,INCIDENT_DATE,TOTAL_OFFENDER_COUNT,OFFENDER_RACE,VICTIM_COUNT,OFFENSE_NAME,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,MULTIPLE_OFFENSE,MULTIPLE_BIAS
0,1991,Rogers,City,AR,Arkansas,West South Central,South,1.3,"Cities from 10,000 thru 24,999",31-Aug-91,1,White,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
1,1991,Hope,City,AR,Arkansas,West South Central,South,1.2,"Cities from 2,500 thru 9,999",19-Sep-91,1,Black or African American,1,Simple Assault,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S
2,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,"Cities from 50,000 thru 99,999",04-Jul-91,1,Black or African American,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
3,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,"Cities from 50,000 thru 99,999",24-Dec-91,1,Black or African American,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
4,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,"Cities from 50,000 thru 99,999",23-Dec-91,1,Black or African American,1,Aggravated Assault,1.0,Service/Gas Station,Anti-White,Individual,S,S
5,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,"Cities from 50,000 thru 99,999",27-Jul-91,1,Black or African American,1,Robbery,1.0,Grocery/Supermarket,Anti-White,Individual,S,S
6,1991,Little Rock,City,AR,Arkansas,West South Central,South,1.6,"Cities from 100,000 thru 249,999",14-Nov-91,2,Black or African American,1,Aggravated Assault,1.0,School/College,Anti-White,Individual,S,S
7,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,"Cities from 50,000 thru 99,999",10-Jul-91,1,Black or African American,2,Aggravated Assault;Murder and Nonnegligent Man...,2.0,Residence/Home,Anti-White,Individual,M,S
8,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,"Cities from 50,000 thru 99,999",06-Oct-91,2,Black or African American,1,Intimidation,1.0,Residence/Home,Anti-White,Individual,S,S
9,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,"Cities from 50,000 thru 99,999",04-Oct-91,10,Black or African American,2,Robbery,2.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S


In [11]:
data.drop(columns=['POPULATION_GROUP_DESC'], inplace=True)

In [12]:
data.head(10)

,DATA_YEAR,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,POPULATION_GROUP_CODE,INCIDENT_DATE,TOTAL_OFFENDER_COUNT,OFFENDER_RACE,VICTIM_COUNT,OFFENSE_NAME,TOTAL_INDIVIDUAL_VICTIMS,LOCATION_NAME,BIAS_DESC,VICTIM_TYPES,MULTIPLE_OFFENSE,MULTIPLE_BIAS
0,1991,Rogers,City,AR,Arkansas,West South Central,South,1.3,31-Aug-91,1,White,1,Intimidation,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-Black or African American,Individual,S,S
1,1991,Hope,City,AR,Arkansas,West South Central,South,1.2,19-Sep-91,1,Black or African American,1,Simple Assault,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S
2,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,04-Jul-91,1,Black or African American,1,Aggravated Assault,1.0,Residence/Home,Anti-Black or African American,Individual,S,S
3,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,24-Dec-91,1,Black or African American,2,Aggravated Assault;Destruction/Damage/Vandalis...,1.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,M,S
4,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,23-Dec-91,1,Black or African American,1,Aggravated Assault,1.0,Service/Gas Station,Anti-White,Individual,S,S
5,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,27-Jul-91,1,Black or African American,1,Robbery,1.0,Grocery/Supermarket,Anti-White,Individual,S,S
6,1991,Little Rock,City,AR,Arkansas,West South Central,South,1.6,14-Nov-91,2,Black or African American,1,Aggravated Assault,1.0,School/College,Anti-White,Individual,S,S
7,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,10-Jul-91,1,Black or African American,2,Aggravated Assault;Murder and Nonnegligent Man...,2.0,Residence/Home,Anti-White,Individual,M,S
8,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,06-Oct-91,2,Black or African American,1,Intimidation,1.0,Residence/Home,Anti-White,Individual,S,S
9,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,04-Oct-91,10,Black or African American,2,Robbery,2.0,Highway/Road/Alley/Street/Sidewalk,Anti-White,Individual,S,S


In [13]:
data['OFFENSE_NAME'] = data['OFFENSE_NAME'].str.split(";")

temp = (
    data['OFFENSE_NAME'].explode()
    .str.get_dummies().sum(level=0)
    .add_prefix('OFFENSE_NAME_')
)

data = data.drop('OFFENSE_NAME', 1).join(temp)

/var/folders/13/mc_783n53h16bdsggstml1_m0000gn/T/ipykernel_28974/321763340.py:4: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  data['OFFENSE_NAME'].explode()
/var/folders/13/mc_783n53h16bdsggstml1_m0000gn/T/ipykernel_28974/321763340.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = data.drop('OFFENSE_NAME', 1).join(temp)


In [14]:
data.head(10)

,DATA_YEAR,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,POPULATION_GROUP_CODE,INCIDENT_DATE,TOTAL_OFFENDER_COUNT,...,OFFENSE_NAME_Sodomy,OFFENSE_NAME_Statutory Rape,OFFENSE_NAME_Stolen Property Offenses,OFFENSE_NAME_Theft From Building,OFFENSE_NAME_Theft From Coin-Operated Machine or Device,OFFENSE_NAME_Theft From Motor Vehicle,OFFENSE_NAME_Theft of Motor Vehicle Parts or Accessories,OFFENSE_NAME_Weapon Law Violations,OFFENSE_NAME_Welfare Fraud,OFFENSE_NAME_Wire Fraud
0,1991,Rogers,City,AR,Arkansas,West South Central,South,1.3,31-Aug-91,1,...,0,0,0,0,0,0,0,0,0,0
1,1991,Hope,City,AR,Arkansas,West South Central,South,1.2,19-Sep-91,1,...,0,0,0,0,0,0,0,0,0,0
2,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,04-Jul-91,1,...,0,0,0,0,0,0,0,0,0,0
3,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,24-Dec-91,1,...,0,0,0,0,0,0,0,0,0,0
4,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,23-Dec-91,1,...,0,0,0,0,0,0,0,0,0,0
5,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,27-Jul-91,1,...,0,0,0,0,0,0,0,0,0,0
6,1991,Little Rock,City,AR,Arkansas,West South Central,South,1.6,14-Nov-91,2,...,0,0,0,0,0,0,0,0,0,0
7,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,10-Jul-91,1,...,0,0,0,0,0,0,0,0,0,0
8,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,06-Oct-91,2,...,0,0,0,0,0,0,0,0,0,0
9,1991,North Little Rock,City,AR,Arkansas,West South Central,South,1.5,04-Oct-91,10,...,0,0,0,0,0,0,0,0,0,0


In [15]:
data['MULTIPLE_OFFENSE'].value_counts()

S    209857
M      9720
Name: MULTIPLE_OFFENSE, dtype: int64

In [16]:
data['MULTIPLE_BIAS'].value_counts()

S    218558
M      1019
Name: MULTIPLE_BIAS, dtype: int64

Since the Multiple Offense and Multiple Bias columns contain only 2 unique categorical values, they are transformed into binary representations.

In [17]:
data['MULTIPLE_OFFENSE'] = data['MULTIPLE_OFFENSE'].replace(["S", "M"], [0,1])
data['MULTIPLE_BIAS'] = data['MULTIPLE_BIAS'].replace(["S", "M"], [0,1])
data.head()

,DATA_YEAR,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,POPULATION_GROUP_CODE,INCIDENT_DATE,TOTAL_OFFENDER_COUNT,...,OFFENSE_NAME_Sodomy,OFFENSE_NAME_Statutory Rape,OFFENSE_NAME_Stolen Property Offenses,OFFENSE_NAME_Theft From Building,OFFENSE_NAME_Theft From Coin-Operated Machine or Device,OFFENSE_NAME_Theft From Motor Vehicle,OFFENSE_NAME_Theft of Motor Vehicle Parts or Accessories,OFFENSE_NAME_Weapon Law Violations,OFFENSE_NAME_Welfare Fraud,OFFENSE_NAME_Wire Fraud
0,1991,Rogers,City,AR,Arkansas,West South Central,South,1.3,31-Aug-91,1,...,0,0,0,0,0,0,0,0,0,0
1,1991,Hope,City,AR,Arkansas,West South Central,South,1.2,19-Sep-91,1,...,0,0,0,0,0,0,0,0,0,0
2,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,04-Jul-91,1,...,0,0,0,0,0,0,0,0,0,0
3,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,24-Dec-91,1,...,0,0,0,0,0,0,0,0,0,0
4,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,23-Dec-91,1,...,0,0,0,0,0,0,0,0,0,0


In [18]:
data['LOCATION_NAME'].value_counts()

Residence/Home                                               65093
Highway/Road/Alley/Street/Sidewalk                           41496
Other/Unknown                                                29351
School/College                                               17479
Parking/Drop Lot/Garage                                      12509
                                                             ...  
Bar/Nightclub;Hotel/Motel/Etc.                                   1
Residence/Home;Service/Gas Station                               1
Convenience Store;Government/Public Building                     1
Air/Bus/Train Terminal;Highway/Road/Alley/Street/Sidewalk        1
Highway/Road/Alley/Street/Sidewalk;Park/Playground               1
Name: LOCATION_NAME, Length: 131, dtype: int64

In [19]:
data['VICTIM_TYPES'].value_counts()

Individual                                               173885
Other                                                     13690
Business                                                   9600
Religious Organization                                     6169
Government                                                 5989
Society/Public                                             5072
Individual;Other                                           1593
Unknown                                                    1316
Business;Individual                                         877
Individual;Society/Public                                   399
Individual;Religious Organization                           223
Government;Individual                                       194
Law Enforcement Officer                                     173
Financial Institution                                        97
Individual;Unknown                                           54
Individual;Law Enforcement Officer      

By calling the value_counts function on the Location Name and Victim Type features we can see that some records are multi-valued. To address this we will perform one hot encoding on both columns.

In [20]:
df_loc_1hot = data['LOCATION_NAME'].str.get_dummies(sep=";")
df_loc_1hot.head()

,ATM Separate from Bank,Abandoned/Condemned Structure,Air/Bus/Train Terminal,Amusement Park,Arena/Stadium/Fairgrounds/Coliseum,Auto Dealership New/Used,Bank/Savings and Loan,Bar/Nightclub,Camp/Campground,Church/Synagogue/Temple/Mosque,...,Rest Area,Restaurant,School-College/University,School-Elementary/Secondary,School/College,Service/Gas Station,Shelter-Mission/Homeless,Shopping Mall,Specialty Store,Tribal Lands
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [21]:
data = pd.concat([data.drop('LOCATION_NAME', 1), df_loc_1hot], 1)
data.shape

/var/folders/13/mc_783n53h16bdsggstml1_m0000gn/T/ipykernel_28974/1587244666.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = pd.concat([data.drop('LOCATION_NAME', 1), df_loc_1hot], 1)
/var/folders/13/mc_783n53h16bdsggstml1_m0000gn/T/ipykernel_28974/1587244666.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  data = pd.concat([data.drop('LOCATION_NAME', 1), df_loc_1hot], 1)


(219577, 111)

In [22]:
df_vic_1hot = data['VICTIM_TYPES'].str.get_dummies(sep=";")
df_vic_1hot.head()

,Business,Financial Institution,Government,Individual,Law Enforcement Officer,Other,Religious Organization,Society/Public,Unknown
0,0,0,0,1,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0


In [23]:
data = pd.concat([data.drop('VICTIM_TYPES', 1), df_vic_1hot], 1)
data.shape

/var/folders/13/mc_783n53h16bdsggstml1_m0000gn/T/ipykernel_28974/3290072729.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data = pd.concat([data.drop('VICTIM_TYPES', 1), df_vic_1hot], 1)
/var/folders/13/mc_783n53h16bdsggstml1_m0000gn/T/ipykernel_28974/3290072729.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  data = pd.concat([data.drop('VICTIM_TYPES', 1), df_vic_1hot], 1)


(219577, 119)

In [24]:
data['BIAS_DESC'] = data['BIAS_DESC'].str.replace(";",",")

In [25]:
data['BIAS_DESC'] = data['BIAS_DESC'].str.split(",")

In [26]:
bias = data['BIAS_DESC'].explode()

In [27]:
oneHoteEncode = pd.crosstab(bias.index, bias)

In [28]:
oneHoteEncode.head()

BIAS_DESC,Bisexual,Gay,Greek,Group,Other),or Transgender (Mixed Group),Anti-American Indian or Alaska Native,Anti-Arab,Anti-Asian,Anti-Atheism/Agnosticism,...,Anti-Native Hawaiian or Other Pacific Islander,Anti-Other Christian,Anti-Other Race/Ethnicity/Ancestry,Anti-Other Religion,Anti-Physical Disability,Anti-Protestant,Anti-Sikh,Anti-Transgender,Anti-White,Unknown (offender's motivation not known)
row_0,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [29]:
data = pd.concat([data, oneHoteEncode], axis=1)

In [30]:
data.head()

,DATA_YEAR,PUB_AGENCY_NAME,AGENCY_TYPE_NAME,STATE_ABBR,STATE_NAME,DIVISION_NAME,REGION_NAME,POPULATION_GROUP_CODE,INCIDENT_DATE,TOTAL_OFFENDER_COUNT,...,Anti-Native Hawaiian or Other Pacific Islander,Anti-Other Christian,Anti-Other Race/Ethnicity/Ancestry,Anti-Other Religion,Anti-Physical Disability,Anti-Protestant,Anti-Sikh,Anti-Transgender,Anti-White,Unknown (offender's motivation not known)
0,1991,Rogers,City,AR,Arkansas,West South Central,South,1.3,31-Aug-91,1,...,0,0,0,0,0,0,0,0,0,0
1,1991,Hope,City,AR,Arkansas,West South Central,South,1.2,19-Sep-91,1,...,0,0,0,0,0,0,0,0,1,0
2,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,04-Jul-91,1,...,0,0,0,0,0,0,0,0,0,0
3,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,24-Dec-91,1,...,0,0,0,0,0,0,0,0,1,0
4,1991,Pine Bluff,City,AR,Arkansas,West South Central,South,1.5,23-Dec-91,1,...,0,0,0,0,0,0,0,0,1,0
